In [1]:
!pip uninstall -y transformers
!pip install transformers --upgrade

Found existing installation: transformers 4.52.3
Uninstalling transformers-4.52.3:
  Successfully uninstalled transformers-4.52.3
  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)


In [2]:
from transformers import pipeline
#משימה 1
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

sentences = {
    "שלילי": "The food was cold, the service was slow, and the place was dirty.",
    "חיובי": "She gave an amazing presentation that impressed everyone in the room."
}

for label, sentence in sentences.items():
    result = sentiment_analysis(sentence)[0]
    predicted = result['label']
    score = result['score']

    print(f"משפט ({label}): {sentence}")
    print(f"זיהוי המודל: {predicted} (ביטחון: {score:.2f})")
    print("------")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


משפט (שלילי): The food was cold, the service was slow, and the place was dirty.
זיהוי המודל: NEGATIVE (ביטחון: 1.00)
------
משפט (חיובי): She gave an amazing presentation that impressed everyone in the room.
זיהוי המודל: POSITIVE (ביטחון: 1.00)
------


In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
#משימה 2
sentences = {
    "פחד": "I heard strange footsteps behind me in the dark alley.",
    "עצב": "I feel lonely and nothing seems to matter anymore.",
    "שמחה": "I finally got the job I always dreamed of!"
}

for emotion, sentence in sentences.items():
    result = classifier(sentence)[0]  # נקבל רשימת רגשות עם ציון לכל אחד
    sorted_result = sorted(result, key=lambda x: x["score"], reverse=True)
    top_emotion = sorted_result[0]

    print(f"משפט ({emotion}): {sentence}")
    print(f"זיהוי המודל: {top_emotion['label']} (ביטחון: {top_emotion['score']:.2f})")
    print("------")


In [ ]:
import numpy as np

In [6]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image
# שלב 1: טען תמונה מהאינטרנט
image_path = "/content/צילום מסך 2025-05-30 115029.png"
image = Image.open(image_path).convert("RGB")

# שלב 2: טען את המודל והמעבד
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# שלב 3: רשימת שאלות
questions = [
    "What is the animal in the image?",                  # שאלה על סוג החיה
    "What color is the cat?",                            # שאלה על צבע
    "How many cats are there?"                           # שאלה על כמות
]

# שלב 4: הרצת שאלות ובדיקת תוצאה
for question in questions:
    encoding = processor(image, question, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    predicted_answer = model.config.id2label[idx]

    print(f"שאלה: {question}")
    print(f"תשובת המודל: {predicted_answer}")
    print("------")


שאלה: What is the animal in the image?
תשובת המודל: cat
------
שאלה: What color is the cat?
תשובת המודל: brown
------
שאלה: How many cats are there?
תשובת המודל: 2
------


In [ ]:
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer
#משימה 4 בחרתי במודל CryptoBERT

# טען את המודל והטוקניזר
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# צור pipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, max_length=64, truncation=True, padding='max_length')

# קבוצה 1: פוסטים מקוריים
post_1 = "see y'all tomorrow and can't wait to see ada in the morning, i wonder what price it is going to be at. 😎🐂🤠💯😴, bitcoin is looking good go for it and flash by that 45k."
post_2 = "alright racers, it’s a race to the bottom! good luck today and remember there are no losers (minus those who invested in currency nobody really uses) take your marks... are you ready? go!!"
post_3 = "i'm never selling. the whole market can bottom out. i'll continue to hold this dumpster fire until the day i die if i need to."
df_posts_1 = [post_1, post_2, post_3]

# קבוצה 2: פוסטים חדשים
alt_post_1 = "The market is crashing and I can't believe I didn't sell earlier. Everything is red."
alt_post_2 = "ETH is showing strong recovery. It might be time to buy the dip and hold."
alt_post_3 = "Crypto is unpredictable. I'm just watching for now."
df_posts_2 = [alt_post_1, alt_post_2, alt_post_3]

# הרצת הסיווג
preds_1 = pipe(df_posts_1)
preds_2 = pipe(df_posts_2)

print("תוצאות עבור קבוצה 1:")
for i, p in enumerate(preds_1):
    print(f"פוסט {i+1}: {p}")

print("\nתוצאות עבור קבוצה 2:")
for i, p in enumerate(preds_2):
    print(f"פוסט {i+1}: {p}")
